#Read Streaming Data from Kafka#

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Configure Kafka bootstrap servers
kafka_bootstrap_servers = "bore.pub:9093"

# Read Kafka data in streaming mode with additional timeout configurations
streaming_df = (spark.readStream
               .format("kafka")
               .option("kafka.bootstrap.servers", kafka_bootstrap_servers)
               .option("subscribe", "music_streaming_events")
               .option("startingOffsets", "earliest")
               .option("kafka.metadata.fetch.timeout.ms", "30000")  # Increase metadata fetch timeout to 30 seconds
               .option("kafka.request.timeout.ms", "60000")         # Increase request timeout to 60 seconds
               .option("kafka.session.timeout.ms", "30000")         # Increase session timeout to 30 seconds
               .load())

# Cast the Kafka message value to a string
raw_streaming_df = streaming_df.select(col("value").cast("STRING").alias("message"))

# Write the raw stream to the console for debugging
query_raw = (raw_streaming_df.writeStream
             .format("console")
             .outputMode("append")
             .trigger(processingTime="10 seconds")  # Process every 10 seconds
             .start())

# Wait for the query to terminate (for debugging)
query_raw.awaitTermination(60)  # Wait up to 60 seconds

False

#Parse the JSON Messages#

In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType

# Define the schema for the JSON data
schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("artist", StringType(), True),
    StructField("genre", StringType(), True),
    StructField("listen_timestamp", StringType(), True),
    StructField("title", StringType(), True),
    StructField("year", IntegerType(), True),
    StructField("sales", DoubleType(), True),
    StructField("streams", DoubleType(), True),
    StructField("downloads", DoubleType(), True),
    StructField("radio_plays", DoubleType(), True),
    StructField("rating", DoubleType(), True)
])

# Parse the JSON data into structured columns
parsed_streaming_df = raw_streaming_df.withColumn("data", from_json(col("message"), schema)).select("data.*")

# Filter out records with null user_id
parsed_streaming_df = parsed_streaming_df.filter(col("user_id").isNotNull())

# Scale the streams column (multiply by 100 million)
parsed_streaming_df = parsed_streaming_df.withColumn("streams", col("streams") * 1e8)

#Compute Average Streams per Genre#

In [0]:
from pyspark.sql.functions import window, current_timestamp

# Compute average streams per genre over a 1-minute tumbling window
windowed_avg_streams = (parsed_streaming_df
                        .groupBy(
                            window(current_timestamp(), "1 minute"),
                            "genre"
                        )
                        .agg({"streams": "avg"})
                        .withColumnRenamed("avg(streams)", "avg_streams"))

# Write the results to an in-memory table for real-time querying
query_avg = (windowed_avg_streams
             .writeStream
             .format("memory")
             .queryName("avg_streams_table")
             .outputMode("complete")
             .start())

# For debugging, display the in-memory table (run this in a separate cell or loop)
spark.sql("SELECT genre, avg_streams FROM avg_streams_table ORDER BY avg_streams DESC LIMIT 20").show()

#Stopping Query#

In [0]:
# Stop the streaming query
query.stop()



---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1719398306228774>, line 2
      1 # Stop the streaming query
----> 2 query.stop()

File /databricks/spark/python/pyspark/sql/streaming/query.py:372, in StreamingQuery.stop(self)
    349 def stop(self) -> None:
    350     """
    351     Stop this streaming query.
    352 
   (...)
    370     False
    371     """
--> 372     self._jsq.stop()

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__call__(self, *args)
   1349 command = proto.CALL_COMMAND_NAME +\
   1350     self.command_header +\
   1351     args_command +\
   1352     proto.END_COMMAND_PART
   1354 answer = self.gateway_client.send_command(command)
-> 1355 return_value = get_return_value(
   1356     answer, self.gateway_client, self.target_id, self.name)
   1358 for temp_arg in temp_args:
   1359     if hasattr